In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}


<IPython.core.display.Javascript object>

In [3]:
IF_SQLITE = True

In [6]:

from CoreConfig.CoreConfig import CoreConfig
if IF_SQLITE:
    from DBUtility.SQLiteUtility import DBSQLiteConnection, CreateEventsTableIfNotExist, CreateVenturesTeamTableIfNotExist
    from DBUtility.SQLiteUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson
    from DBUtility.SQLiteUtility import FetchEventsRecordAll, FetchVenturesPerson, InsertVenturesPerson
    from DBUtility.SQLiteUtility import InsertEventRecord
    from DBUtility.SQLiteUtility import FetchVenturesEngagement, CreateVenturesEngagementTableIfNotExist
else:
    from DBUtility.DBUtility import DBConnection, CreateEventsTableIfNotExist, CreateVenturesTeamTableIfNotExist
    from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson
    from DBUtility.DBUtility import FetchEventsRecordAll, FetchVenturesPerson, InsertVenturesPerson
    from DBUtility.DBUtility import InsertEventRecord
    from DBUtility.DBUtility import FetchVenturesEngagement, CreateVenturesEngagementTableIfNotExist
from pandas import DataFrame
import datetime
CoreConfig.initialize_core_config()
if IF_SQLITE:
    DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
else:
    DBConnection().configure_connection_information(**CoreConfig().get_db_connection_info())

CreateEventsTableIfNotExist().execute()

CreateVenturesTeamTableIfNotExist().execute()

CreateVenturesEngagementTableIfNotExist().execute()

Initialize nab ventures events board             table failed. Please Check your connection
table nab_ventures_events_board already exists
Initialize Ventures team table            failed. Please Check your connection
table NAB_VENTURES_TEAM already exists
Initialize Ventures engagement table            failed. Please Check your connection
table nab_ventures_engagement_types already exists


In [7]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
items_layout = Layout(display='flex',
                    flex_flow='auto',
                     align_items='stretch',
                      width='auto')     # override the default width of the button to 'auto' to let the button grow



box_layout = Layout(display='flex',
                    flex_flow='auto',
                    align_items='center',
                    width='auto')

textarea_layout = Layout(display='flex',
                    flex_flow='auto',
                    align_items='center',
                    width='auto',
                    height='120px')

style = {'description_width': 'initial'}

In [17]:
title = widgets.HTML("<center><H3>Recent records</H3><center>")
formtitle = widgets.HTML("<center><H3>Submit Form</H3><center>")
df_out = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})
with df_out:
    df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    display(df)
    
events = widgets.Text(
      value='',
      placeholder='Type Events',
      description='Events:',
      layout=box_layout,
       style=style,
      disable=False)

date = widgets.DatePicker(
        value = datetime.datetime.now().date(),
        description='Pick a Date',
        layout=box_layout,
        style = style,
        disable=False)

engagement_list = FetchVenturesEngagement().execute()
engagement_dict = {v[0]:v[0] for i, v in enumerate(engagement_list)}


engagement = widgets.Dropdown(options= engagement_dict,
                               description='Engagement Type:',
                               layout=box_layout,
                               style=style)
#display_pretty(name)
notes=widgets.Text(
      value='',
      placeholder='Type note/organizer',
      description='Note/ Organizer:',
      layout=box_layout,
       style=style,
      disable=False)

country = widgets.Dropdown(options= {'Australia': 'Australia',
                                     'New Zealand': 'New Zealand',
                                    'None':'None'},
                           value = 'Australia',
                           description='Select country:',
                           layout=box_layout,
                           style=style)
location= widgets.Text(value='',
                       placeholder='Type the location',
                       description='Location:',
                       layout=items_layout,
                       style=style)

audience= widgets.Text(value='0',
                       placeholder='Type audience numbers',
                       description='Audience:',
                       layout=box_layout,
                       style=style)

quality = widgets.Dropdown(options= {'1': '1',
                                     '2': '2',
                                    '3':'3','4':'4','5':'5'},
                           value = '3',
                           description='Rate the event:',
                           layout=box_layout,
                           style=style
)

website= widgets.Text(value='https://',
                       placeholder='Type website/context',
                       description='Website:',
                       layout=box_layout,
                       style=style,
                       disable=False)
asset = widgets.Text(value='',
                       placeholder='Type asset',
                       description='Asset:',
                       layout=box_layout,
                       style=style,
                       disable=False)


person_list = FetchVenturesPerson().execute()
person_dict = {v[0]:v[0] for i, v in enumerate(person_list)}
person = widgets.Dropdown(
                options= person_dict,
                description='Select your person:',
                layout=box_layout,
                style=style
              )

new_person_check = widgets.Checkbox(
    value=False,
    description='Add new person',
    disable = False
)
new_person_text = widgets.Text(
      value='',
      placeholder='Type your name',
      description='New person to be added:',
      layout=box_layout,
       style=style,
      disabled=True)

new_person_button = widgets.Button(
    description='Submit',
    disabled=True,
    button_style ='', # success
    tooltip='Click me',
    icon='check')



new_person_box = widgets.HBox([new_person_check, new_person_text, new_person_button])

button = widgets.Button(
           description='Submit the form',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_add_new_name_value_change(change):
    if change['new'] == True:
        new_person_text.disabled=False
        new_person_button.disabled=False
    else:
        new_person_text.disabled=True
        new_person_button.disabled=True


def on_submit_add_new_name_button_clicked(b):
    '''Check the local person_dict, if not included, 
    then added and refresh
    '''
    temp_variable = person.options.copy()
    new_person = new_person_text.value
    temp_variable[new_person] = new_person
    person.options = temp_variable
    new_person_check.value = False
    new_person_text.disabled=True
    new_person_button.disabled=True
    InsertVenturesPerson(new_person).execute()

new_person_check.observe(on_add_new_name_value_change, names='value')
new_person_button.on_click(on_submit_add_new_name_button_clicked)
def on_submit_form_button_clicked(b):
    InsertEventRecord(events.value, date.value, engagement.value, notes.value,
                      country.value, location.value, quality.value, audience.value, website.value, asset.value, person.value).execute()
    df = DataFrame(FetchEventsLatestRecordWithSamePerson().execute(), columns=['Events Name', 'Person', 'Date'])
    df_out.clear_output()
    with df_out:
        display(df)
button.on_click(on_submit_form_button_clicked)

In [18]:
def form(events, date, engagement, notes, country, location, audience,quality, website, asset, person, new_person_name):
    #display(events, meeting, date, name, notes, country, audience, website, asset, person)
    return events, date, engagement, notes, country, location, audience,quality, website, asset, person, new_person_name

In [19]:
ui = widgets.VBox(children=[title,
                            df_out,
                            formtitle,
                            person,
                            widgets.HBox(children=[events,date], layout=items_layout),
                            engagement,
                            notes,
                            widgets.HBox(children=[country], layout=items_layout),
                            location,
                            audience,
                            quality,
                            website,
                            asset,
                            button,new_person_box]
                            , layout=items_layout)

In [20]:
now = datetime.datetime.now()
now.strftime("%Y%m")

'201809'

In [21]:
out = interactive_output(form, {'events': events,
                                'date':date,
                                'engagement':engagement,
                                'notes':notes,
                                'country':country,
                                'location':location,
                                'audience': audience,
                                'quality' : quality,
                                'website': website,
                                'asset' : asset,
                                'person' : person,
                                'new_person_name' : new_person_box.children[1]
                               })
display(ui, out)


Output()